Este notebook mostra como realizar busca semântica utilizando o modelo [sentence-transformer-ult5-pt-small](https://huggingface.co/tgsc/sentence-transformer-ult5-pt-small)

# Instalação dos pacotes

In [1]:
!pip install datasets sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 117.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.9 MB/s eta 

# Download do dataset

In [2]:
!wget https://github.com/thacio/Few-Shot-Editais-TI/raw/main/editais_TI.xlsx

--2023-05-21 15:22:42--  https://github.com/thacio/Few-Shot-Editais-TI/raw/main/editais_TI.xlsx
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thacio/Few-Shot-Editais-TI/main/editais_TI.xlsx [following]
--2023-05-21 15:22:42--  https://raw.githubusercontent.com/thacio/Few-Shot-Editais-TI/main/editais_TI.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 702524 (686K) [application/octet-stream]
Saving to: ‘editais_TI.xlsx’

editais_TI.xlsx     100%[===================>] 686.06K  --.-KB/s    in 0.004s  

2023-05-21 15:22:42 (160 MB/s) - ‘editais_TI.xlsx’ saved [702524/702524]



O dataset consiste em objetos de editais de licitações, classificados em 'Tecnologia de informação' e 'Outras compras', caso não sejam de TI.

Esta célula carrega o dataset em pandas e elimina as linhas repetidas

In [3]:
import pandas
import numpy

pandas.set_option('display.max_colwidth', None)

df = pandas.read_excel('editais_TI.xlsx')
df['CLASSE_TI'] = df['CLASSE_TI'].replace({0: 'Outras compras', 1: 'Tecnologia da Informação'})
df['DES_OBJETO'] = df['DES_OBJETO'].str.replace('Objeto: Pregão Eletrônico - ', '').str.strip()
df = df[~df['DES_OBJETO'].duplicated()] # Remove colunas com objeto duplicado

df.head()

,Unnamed: 0,COD_UASG,NUM_ANO_PREGAO,DES_OBJETO,CLASSE_TI
0,0,153038,12019,"O objeto da presente licitação é de empresa para realização de serviços técnicos especializados na área de tecnologia da informação, contemplando a implantação e execução continuada das atividades de Gerenciamento e Operação da Central de Serviços de TIC de 1º, 2º e 3º níveis, para atender às necessidades da Universidade Federal da Bahia, em seus diversos órgãos e unidades situados nas cidades de Salvador, Vitória da Conquista, Oliveira dos Campinhos (Santo Amaro) e Camaçari, conforme condições, quantidades e exigências estabelecidas neste Edital e seus anexos.",Outras compras
1,1,153054,122019,"Registro de Preços para futuras aquisições de material farmacológico (anastrozol, cisplatina, citarabina e outros) para o Hospital das Clínicas-UFG/EBSERH",Outras compras
2,2,153054,152019,Registro de preços para futuras aquisições de material farmacológico (carboplatina 150 mg e outros).,Outras compras
3,3,153054,192019,Contratação de empresa para prestação de serviço especializado de portaria com dedicação de mão de obra exclusiva.,Outras compras
4,4,153054,332019,Registro de preço para futuras aquisições de Órtese e Prótese (Cardiodesfibrilador com marca passo multi-sitio e outros).,Outras compras


# Cálculo dos embeddings

##Carrega o modelo

utilizando a biblioteca sentence transformers

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('tgsc/sentence-transformer-ult5-pt-small')

## Calcula os embeddings

dos objetos de licitação na coluna 'DES_OBJETO' e salva na coluna 'EMBEDDINGS'

In [5]:
# %%time
# df['EMBEDDINGS'] = df['DES_OBJETO'].apply(lambda x: model.encode(x,convert_to_tensor=True).cpu())

faz o mesmo procedimento, porém usando batchs para acelerar caso se utilize gpu

In [6]:
%%time
sentences = df['DES_OBJETO'].tolist()
batch_size = 512  # Adjust batch size as per your requirements
batches = [sentences[i:i+batch_size] for i in range(0, len(sentences), batch_size)]
embeddings = []

for batch in batches:
    batch_encodings = model.encode(batch,convert_to_tensor=True).cpu()
    embeddings.extend(batch_encodings)

df['EMBEDDINGS'] = embeddings

CPU times: user 9.43 s, sys: 1.16 s, total: 10.6 s
Wall time: 18 s


# Realiza a busca semântica.

1. Define o texto que será pesquisado
2. Calcula os embeddings do texto de pesquisa
3. Realiza o cálculo da similaridade por cosseno do texto a ser pesquisado com cada item do dataset
4. Ordena em ordem decrescente (quanto maior a similaridade por cosseno, mais próximo é a pesquisa)

## Busca por área de TI

texto: 'Compras relacionadas à area de Tecnologia da informação, softwares e materiais de informática'

In [7]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à área de Tecnologia da informação, softwares e materiais de informática'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

# df = df.drop('SIMILARIDADE_COS', axis=1)
df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

CPU times: user 1.93 s, sys: 3.36 ms, total: 1.93 s
Wall time: 1.97 s


In [8]:
df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('TI.xlsx')

Vamos contar qual a porcentagem dos 100 primeiros itens estão classificados como Tecnologia da informação

In [9]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 99
Percentage: 99.0 %


99% dos 100 primeiros elementos estão classificados como TI.
Vamos dar uma olhada no único que estão classificados como 'Outras compras'

In [10]:
subset[subset['CLASSE_TI'] == 'Outras compras']['DES_OBJETO'].tolist()

['Prestação de serviços técnicos comuns de telecomunicações e eletrônica de infraestrutura de dados.']

É possível ver que o resultado é pode ser considerado de TI.

Os 20 melhores resultados

In [11]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(20)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
757,Aquisição de equipamentos de Tecnologia da Informação.,Tecnologia da Informação,0.768308
5008,"Contratação de serviços de tecnologia da informação na área de sustentação de sistemas de informação, de acordo com o planejamento de TI e em apoio à Coordenação de Tecnologia da Informação e Comunicação do LNCC.",Tecnologia da Informação,0.752999
1454,"Contratação de empresa especializada para a prestação de serviços técnicos e continuados na área de tecnologia da informação, com transferência de conhecimento, para suportar as atividades de desenvolvimento e operações de soluções de Tecnologia da Informação e Comunicação (TIC)",Tecnologia da Informação,0.740502
3481,Eventual aquisição de materiais de consumo de informática e comunicações.,Tecnologia da Informação,0.721085
3010,Aquisição de equipamentos de tecnologia da informação (tic),Tecnologia da Informação,0.720201
1539,Aquisição de materiais de informática.,Tecnologia da Informação,0.718420
5911,Contratação de empresa para o fornecimento de diversos equipamentos e materiais de Tecnologia da Informação.,Tecnologia da Informação,0.717873
3589,Aquisição de equipamentos e materiais de informática para suprir as necessidades da Seção de Tecnologia da Informação da 18ª Bda Inf Fron e 18ª Companhia de Comunicações.,Tecnologia da Informação,0.715141
4720,"Contratação de empresa especializada para a prestação de serviços técnicos continuados de suporte tecnológico ao ambiente computacional, contemplando a sustentação e operação, controle operacional e qualidade e serviços especializados de TI da infraestrutura de Tecnologia da Informação.",Tecnologia da Informação,0.709799
2562,"Aquisição de computadores, outros equipamentos de informática, bens permanentes e bens de consumo de informática",Tecnologia da Informação,0.702529


Os 10 últimos resultados

In [12]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
4391,"Radiador do óleo hidráulico modelo 900/200 PCM, para compressor de ar marca GARDNEN DENVER, modelo SP 750.",Outras compras,0.075659
5076,Cassete de peróxido de hidrogênio a 56% e outros.,Outras compras,0.072089
4882,Ácido Tranexâmico 250 mg,Outras compras,0.068446
6342,Carne Bovina,Outras compras,0.063492
6608,Solução de hidrazina,Outras compras,0.056052
4161,Furadeiras de furação profunda.,Outras compras,0.053804
6207,Óleo Diesel Marítimo,Outras compras,0.045121
4160,Óleo Desengripante e Araldite,Outras compras,0.039802
6613,Óleo Lubrificante,Outras compras,0.022952
5519,Gás refrigerante,Outras compras,0.000946


## Busca por área de Educação 

texto: 'Compras relacionadas à área de educação, escolas e ensino.'

Agora vamos pesquisar compras relacionadas à área de educação

In [13]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à área de educação, escolas e ensino.'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

# df = df.drop('SIMILARIDADE_COS', axis=1)
df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('educação.xlsx')

CPU times: user 3.65 s, sys: 20.7 ms, total: 3.67 s
Wall time: 3.69 s


Agora vamos ver quanto dos 100 primeiros estão classificados como 'Tecnologia da informação', e se estão, quais são os objetos

In [14]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 6
Percentage: 6.0 %


In [15]:
subset[subset['CLASSE_TI'] == 'Tecnologia da Informação']['DES_OBJETO'].tolist()

['Registro de preços para aquisição de conjuntos de robótica educacional destinados às escolas públicas estaduais, municipais e do Distrito Federal.',
 'Aquisição de MATERIAIS PARA LOGÍSTICA E ESTRUTURAÇÃO DE ATIVIDADES DE ENSINO, PESQUISA, EXTENSÃO E INTERNACIONALIZAÇÃO, DENTRE OUTROS, ESPECIFICAMENTE PARA OS RESPECTIVOS PLANOS DE COMUNICAÇÃO DE EVENTOS ACADÊMICOS, CIENTÍFICOS, ADMINISTRATIVOS E MISSÕES OFICIAIS por esta SCS/UFPI e seus quatro órgãos vinculados que atendem a TODOS OS CAMPI DA UNIVERSIDADE FEDERAL DO PIAUÍ, em atividades de ensino, pesquisa, extensão, internacionalização e missões oficiais',
 'Aquisição de materiais de consumo de informática para o HCE.',
 'REGISTRO DE PREÇOS visando a aquisição de suprimentos de informática.',
 'Prestação de serviços técnicos de consultoria e assessoramento especializado em finanças corporativas, visando dar suporte técnico, metodológico e operacional em todos os processos de modelagem econômico-financeira de FURNAS CENTRAIS ELÉTRICAS

Tivemos quatro que não teriam relação com a área de educação, sendo os demais pertinentes.

---

Os 10 melhores resultados

In [16]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
4142,"Registro de preços para futura e eventual aquisição de brindes e premiações, para atender as necessidades da Escola de Formação Complementar e Colégio Militar de Salvador e as UG participantes do Grupo de Coordenação e Acompanhamento das Licitações (GCAL) da 6º RM.",Outras compras,0.580732
3046,"Contratação de empresa especializada para prestação de serviço de Recepção, nas dependências da Reitoria, do Polo de Inovação, do Centro de Referência em Formação e Educação a Distancia e demais Campi do Ifes.",Outras compras,0.567480
2285,"Registro de preços com vistas a aquisição de bicicleta e capacete escolar, em atendimento às entidades educacionais das redes públicas de ensino nos Estados, Distrito Federal e Municípios.",Outras compras,0.557586
2280,"Registro de preços para aquisição de conjuntos de robótica educacional destinados às escolas públicas estaduais, municipais e do Distrito Federal.",Tecnologia da Informação,0.557485
2479,"O objeto da presente licitação é o registro de preços para Contratação de editora(s) e/ou distribuidor(as) para fornecimento de publicações nacionais e importadas, tipo livros, DVDs, CDs rom e/ou audiovisuais, em diferentes áreas de conhecimento (Ciências Sociais e Aplicadas, Ciências Humanas, Linguística, Letras e Artes, Ciências Exatas, da Terra, Tecnologia e Engenharias, Ciências Biológicas, da Saúde e Agrárias).",Outras compras,0.553673
3220,"Aquisição parcelada e futura de ACHOCOLATADO E OUTROS, pelo SISTEMA DE REGISTRO DE PREÇOS (SRP) necessários para o preparo da merenda escolar dos alunos do Campus Campos-Centro do INSTITUTO FEDERAL FLUMINENSE.",Outras compras,0.550700
2527,"Aquisição de MATERIAIS PARA LOGÍSTICA E ESTRUTURAÇÃO DE ATIVIDADES DE ENSINO, PESQUISA, EXTENSÃO E INTERNACIONALIZAÇÃO, DENTRE OUTROS, ESPECIFICAMENTE PARA OS RESPECTIVOS PLANOS DE COMUNICAÇÃO DE EVENTOS ACADÊMICOS, CIENTÍFICOS, ADMINISTRATIVOS E MISSÕES OFICIAIS por esta SCS/UFPI e seus quatro órgãos vinculados que atendem a TODOS OS CAMPI DA UNIVERSIDADE FEDERAL DO PIAUÍ, em atividades de ensino, pesquisa, extensão, internacionalização e missões oficiais",Tecnologia da Informação,0.547798
3186,Eventual aquisição de merenda escolar industrializada para os discentes do IFCE Campus Tianguá e demais órgãos participantes.,Outras compras,0.542280
4324,"Contratação de pessoa jurídica especializada na prestação de serviços de AGENTE DE INTEGRAÇÃO DE ESTÁGIO (INTERMEDIAÇÃO PARA SELEÇÃO, CONTRATAÇÃO E ACOMPANHAMENTO DE ESTAGIÁRIOS), visando atender aos estudantes de educação superior e de ensino médio, vinculados à estrutura do ensino público e privado do país, para preenchimento de oportunidades de estágio no âmbito da SAMF/PB e de órgãos por ela atendidos, conforme as especificações técnicas e condições constantes do Edital e seus Anexos.",Outras compras,0.540212
2593,Eventual aquisição de bens de consumo e permanentes de TI e eletrônica para o Campus Poços de Caldas - IFSULDEMINAS.,Outras compras,0.538482


Os 10 piores

In [17]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
5230,Filtros de HVAC.,Outras compras,0.060727
3438,Mamógrafo,Outras compras,0.056098
4160,Óleo Desengripante e Araldite,Outras compras,0.052851
6207,Óleo Diesel Marítimo,Outras compras,0.052838
6608,Solução de hidrazina,Outras compras,0.051848
4161,Furadeiras de furação profunda.,Outras compras,0.044134
600,Sistema de Aeronave Remotamente Pilotada (RPAS),Tecnologia da Informação,0.040479
6593,Reparo do reator de derivacao monofasico tusa 550 kv da subestacao messias,Outras compras,0.034976
6463,Retificadores,Outras compras,-0.010962
5519,Gás refrigerante,Outras compras,-0.050793


## Busca por área de Saúde

texto: 'Compras relacionadas à área de saúde, hospitais e remédios.'

In [18]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à área de saúde, hospitais e remédios.'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('saúde.xlsx')

CPU times: user 3.64 s, sys: 41.3 ms, total: 3.68 s
Wall time: 3.72 s


Agora vamos ver quanto dos 100 primeiros estão classificados como 'Tecnologia da informação', e se estão, quais são os objetos

In [19]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 0
Percentage: 0.0 %


In [20]:
subset[subset['CLASSE_TI'] == 'Tecnologia da Informação']['DES_OBJETO'].tolist()

[]

Os 10 melhores resultados

In [21]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
2204,Aquisição de Produtos para Saúde para atender ao Serviço de Farmácia.,Outras compras,0.736511
2981,Aquisição de produtos para saúde.,Outras compras,0.732564
1873,Registro de preços visando a eventual aquisição de Material Médico Hospitalar - Cardiologia Intervencionista para o Hospital das Clínicas da UFPE,Outras compras,0.716888
2687,Produtos para Saúde,Outras compras,0.715993
1897,Registro de preços visando a eventual aquisição de Medicamentos (Soluções Parenterais de Grande Volume) para o Hospital das Clínicas da UFPE,Outras compras,0.714513
1883,Registro de preços visando a eventual aquisição de Medicamentos para o Hospital das Clínicas da UFPE,Outras compras,0.713425
3204,"Aquisição de Material de Consumo Médico Hospitalar - (Equipos, Extensores, etc.)",Outras compras,0.713065
1899,Registro de preços visando a eventual aquisição de Medicamentos Gerais para o Hospital das Clínicas da UFPE.,Outras compras,0.712226
626,Registro de preços para aquisição de Material de consumo de Saúde (Medicamentos) para o HOSPITAL DAS FORÇAS ARMADAS. PROCESSO (60550.016640/2018-74),Outras compras,0.711958
2030,Registro de preços visando a eventual aquisição de Medicamentos para o Hospital das Clínicas da UFPE.,Outras compras,0.709073


Os 10 piores resultados

In [22]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
6207,Óleo Diesel Marítimo,Outras compras,0.092492
6593,Reparo do reator de derivacao monofasico tusa 550 kv da subestacao messias,Outras compras,0.090659
6142,Trator Multiuso e Manipulador Telescócipo,Outras compras,0.087895
4161,Furadeiras de furação profunda.,Outras compras,0.087167
4786,Capacete balístico nível III-A,Outras compras,0.082488
6594,Transformador de excitacao,Outras compras,0.069873
4468,"Coldre tático para pistolas Glock G17, canhotos ou destros.",Outras compras,0.068323
3994,Formação de condutores.,Outras compras,0.066036
6602,Isoladores polimericos,Outras compras,0.046475
6463,Retificadores,Outras compras,0.018546


## Busca por área de Construção civil

texto: 'Compras relacionadas à área de construção civil, obras e rodovias.'

In [23]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à área de construção civil, obras e rodovias.'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

# df = df.drop('SIMILARIDADE_COS', axis=1)
df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('construção_civil.xlsx')

CPU times: user 3.15 s, sys: 13.6 ms, total: 3.16 s
Wall time: 3.18 s


Agora vamos ver quanto dos 100 primeiros estão classificados como 'Tecnologia da informação', e se estão, quais são os objetos

In [24]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 0
Percentage: 0.0 %


In [25]:
subset[subset['CLASSE_TI'] == 'Tecnologia da Informação']['DES_OBJETO'].tolist()

[]

Os 10 melhores resultados

In [26]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
3876,"Contratação de empresas para prestar serviços de locação de equipamentos de engenharia, viaturas, transporte de materiais e fornecimento de concreto usinado.",Outras compras,0.670003
868,Contratação de serviços comuns de engenharia de manutenção e conservação de bens imóveis.,Outras compras,0.638142
3255,Aquisição de Material de Construção Civil.,Outras compras,0.635781
4622,"Eventual contratação de empresa de engenharia especializada na prestação serviços, sob demanda, de manutenção predial com fornecimento de materiais, equipamentos e mão de obra, na forma estabelecida nas planilhas de serviços e insumos diversos descritos no Sistema Nacional de Pesquisa de Custos e Índices da Construção Civil, doravante denominado SINAPI, nas áreas, instalações e infraestruturas das unidades da Policia Rodoviária Federal em Roraima.",Outras compras,0.631339
3557,"Registro de preços para eventual contratação de empresa de engenharia para prestar, sob demanda, serviços de alvenaria diversos e serviços de manutenção predial preventiva e corretiva com fornecimento de peças, equipamentos, materiais e mão de obra, na forma estabelecida em planilhas de serviços e insumos diversos descritos no Sistema Nacional de Pesquisa de Custos de índices de Construção Civil.",Outras compras,0.628906
5823,"Contratação de empresa especializada na elaboração de projeto executivo de engenharia para recuperação e restauração do pavimento; recuperação dos aterros, taludes e passivos ambientais e alargamento de acostamentos na rodovia BR-259/ES km 0,0 ao km 106,3.",Outras compras,0.628028
5705,"Contratação de empresa especializada para Execução dos serviços de Supervisão das Obras e Supervisão Ambiental da Implantação, Construção e Pavimentação do Contorno de Aragarças/GO na Rodovia BR-070/GO.",Outras compras,0.619615
1208,Registro de preços para eventual aquisição de materiais de construção e afins.,Outras compras,0.614219
3094,"Registro de preços para eventual contratação de empresa especializada, para sob demanda, prestar de serviços comuns de engenharia inerentes à manutenção predial preventiva e corretiva dos bens imóveis, com fornecimento de peças, equipamentos, materiais e mão de obra, na forma estabelecida nas planilhas de serviços e insumos diversos descritos no Sistema Nacional de Pesquisa de Custos e Índices da Construção Civil, doravante denominada SINAPI.",Outras compras,0.609894
3538,"Contratação de empresas para sob demanda, prestar serviços comuns de engenharia inerentes a manutenção predial com fornecimento de peças, equipamentos, materiais e mão de obra, na forma estabelecida nas planilhas de serviços e insumos diversos descritos no Sistema Nacional de Pesquisa de Custos e Índices da Construção Civil, doravante denominada SINAPI, nas edificações sob responsabilidade da Prefeitura Militar de Brasília, doravante denominada PMB, conforme Edital",Outras compras,0.608343


Os 10 piores resultados

In [27]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
5390,Tomógrafo Computadorizado Multidetector de 16 canais,Outras compras,0.053366
5231,Conjunto de filtração em vidro borosilicado alta resistência e outros.,Outras compras,0.051042
4786,Capacete balístico nível III-A,Outras compras,0.049867
5241,"Sílica gel em sache de 100g , como indicador de umidade acondicionado, individualmente, a vácuo em saco plástico.",Outras compras,0.043941
5239,"Cloroquina, difosfato",Outras compras,0.043610
6411,Solução de filtro de conteúdo Web,Tecnologia da Informação,0.030983
6159,Medalha Militar com Estojo e Medalha Mérito Marinheiro,Outras compras,0.029414
6471,Resina trocadora íons,Outras compras,0.027748
5519,Gás refrigerante,Outras compras,0.003296
6463,Retificadores,Outras compras,-0.029322


## Busca por área indígena

texto: 'Licitações relacionadas à grupos indígenas'

In [28]:
%%time
from sentence_transformers import util

texto_pesquisa='Compras relacionadas à comunidades indígenas'

#Compute embedding for both lists
embeddings_pesquisa = model.encode(texto_pesquisa, convert_to_tensor=True)

# df = df.drop('SIMILARIDADE_COS', axis=1)
df['SIMILARIDADE_COS']=0.0
for idx, row in df.iterrows():
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings_pesquisa.cpu(), row['EMBEDDINGS'])
    df.at[idx, 'SIMILARIDADE_COS'] = cosine_scores[0].item()

df_salvo = df.drop('EMBEDDINGS', axis=1).sort_values(by='SIMILARIDADE_COS', ascending=False)
df_salvo.to_excel('indígena.xlsx')

CPU times: user 2.98 s, sys: 8.48 ms, total: 2.99 s
Wall time: 2.99 s


Agora vamos ver quanto dos 100 primeiros estão classificados como 'Tecnologia da informação', e se estão, quais são os objetos

In [29]:
subset = df_salvo[0:100]
count = len(subset[subset['CLASSE_TI'] == 'Tecnologia da Informação'])  # Count occurrences of 'Tecnologia da informação'

percentage = (count / 100) * 100  # Calculate the percentage

print("Count:", count)
print("Percentage:", percentage, "%")

Count: 2
Percentage: 2.0 %


Os 10 melhores resultados

In [30]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).head(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
4562,"Contratação de empresa(s) especializada(s) no fornecimento de Gêneros Alimentícios e Outros Produtos, de forma parcelada e de acordo com as demandas de projetos e ações, destinados a atender a Coordenação Regional , Coordenações Técnicas Locais e as ações da Comunidade Indígena Avá Canoeiro.",Outras compras,0.694057
4414,"Aquisição de gêneros alimentícios e materiais diversos destinados aos indígenas das diversas etnias jurisdicionados à Coordenação Regional do Maranhão, às Coordenações Técnicas Locais subordinadas a esta e à Frente de Proteção Etnoambiental-FPEA, que residem nas Terras Indígenas dos municípios de Arame, Amarante, Montes Altos, Grajaú, Jenipapo dos Vieiras, Barra do Corda, Santa Inês, Zé Doca e São Luís.",Outras compras,0.631062
4569,"Registro de preços para a aquisição eventual e futura de ferramentas e insumos agrícolas, de equipamentos para a produção de farinha-de-mandioca, de material de pesca e de equipamentos para geração de energia solar, conforme especificações e quantidades constantes neste Termo de Referência, para apoiarem e fomentarem as atividades produtivas e extrativistas tradicionais das comunidades indígenas jurisdicionadas à Coordenação Regional Kayapó Sul do Pará.",Outras compras,0.617561
4574,"Aquisição de gêneros alimentícios com objetivo de suprir a demanda desta Coordenação Regional e Coordenações Técnicas Locais jurisdicionadas, para promover e/ou apoiar a realização de reuniões, seminários, conferências, oficinas, ações de fiscalização e monitoramento ambiental e territorial nas Terras Indígenas, e demais ações a serem desenvolvidas ou apoiadas pela FUNAI, no âmbito da Coordenação Regional Noroeste do Mato Grosso.",Outras compras,0.606893
5475,"Contratação de pessoas jurídicas especializadas para prestação de serviços de fornecimento de hospedagem e de transporte intermunicipal para os participantes do VI Encontro Estadual das Comunidades Quilombolas do Estado de Minas Gerais e I Festival de Cultura Quilombola de Minas Gerais - Projeto CANJERÊ , a ser realizado na cidade de Belo Horizonte-MG, de acordo Edital e Termo de Referência.",Outras compras,0.588408
4575,"Contratação de empresa especializada para prestação de serviços de locação de veículos tipos, da forma menor preço por diária, tendo como finalidade o atendimento às necessidades da Fundação Nacional do Índio FUNAI/Coordenação Regional do Maranhão, Coordenações Técnicas Locais, Frente de Proteção Etno-Ambiental Awá-Guajá, órgãos parceiros da FUNAI e/ou ainda outras Coordenações Regionais quando da realização de trabalhos em parceria com esta Coordenação.",Outras compras,0.567927
5435,"Contratação de empresa especializada no fornecimento com instalação de Rádios Transceptores, para dar suporte às ações de comunicação nas comunidades Indígenas atendidas pelo Distrito Sanitário Especial Indígena Leste de Roraima.",Outras compras,0.561052
1849,"Contratação de Empresa especializada para prestação de serviço de hospedagem, incluindo refeições (05 refeições Café da manhã, almoço, dois lanches e jantar) e espaços adequados para aulas práticas que receberão diversas intervenções físicas no meio ambiente sob os princípios da Agroecologia e utilizando as tecnologias de Convivência com o Semiárido, práticas agroflorestais, cultivo de hortaliças, formação de Farmácias Vivas, Quintais Produtivos...",Outras compras,0.558203
3952,"Eventual aquisição de gêneros de alimentação, tipo hortifrutigranjeiros, em proveito do Setor de Aprovisionamento.",Outras compras,0.557068
4415,"Contratação de empresa especializada no fornecimento de gêneros alimentícios e materiais diversos, fim atender a demanda da CR Maranhão, CTLS e Frente de Proteção Awá.",Outras compras,0.556932


Os 10 piores resultados

In [31]:
df_salvo.drop(['Unnamed: 0', 'COD_UASG', 'NUM_ANO_PREGAO'], axis=1).tail(10)

,DES_OBJETO,CLASSE_TI,SIMILARIDADE_COS
4391,"Radiador do óleo hidráulico modelo 900/200 PCM, para compressor de ar marca GARDNEN DENVER, modelo SP 750.",Outras compras,0.070322
3438,Mamógrafo,Outras compras,0.069698
6461,"Renovacao de licencas windows server, sql server e remote desktop",Tecnologia da Informação,0.065876
1162,Serviço de confecção e instalação de sistemas de arquivamento deslizante e implantação de projeto para segurança contra incêndio.,Outras compras,0.054059
4468,"Coldre tático para pistolas Glock G17, canhotos ou destros.",Outras compras,0.048006
4976,Mesa cirúrgica eletro - hidráulica.,Outras compras,0.016288
4786,Capacete balístico nível III-A,Outras compras,0.013896
6463,Retificadores,Outras compras,0.012452
600,Sistema de Aeronave Remotamente Pilotada (RPAS),Tecnologia da Informação,0.004189
5519,Gás refrigerante,Outras compras,-0.042081
